### Luca's example's start from here

In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as pyf
from pyspark.sql.functions import udf, to_json, from_json, explode, explode_outer, col, array, array_contains, max as max_, min as min_, collect_list, collect_set, arrays_zip, lit, expr, sha2, concat_ws, size, struct, current_timestamp, current_date, sha2, concat_ws
from pyspark.sql.types import Row, ArrayType, StructType, MapType, StructField, StringType, TimestampType, DateType, IntegerType, LongType, DoubleType, BooleanType
import datetime as dt

In [0]:
path = "abfss://process@xsegglobaldistdevadlsa.dfs.core.windows.net/GLOBAL_XSEG_SRM_DATA_FOUNDATION/DEV/DATA_ASSET/DISTRIBUTOR/AMG_POC/"
DIM_COUNTRY = 'dim_country'
DIM_DISTRIBUTOR = 'dim_distributor'
DIM_POS = 'dim_pos'
FACT_SELLOUT = 'fact_sellout'

In [0]:
df_dim_country = spark.createDataFrame(
        [
            ("AU", "AUS", "AUSTRALIA", "AUD", "AUD", "", ""),
            ("NL", "NLD", "NETHERLANDS", "EUR", "EUR", "", ""),
            ("EG", "EGY", "EGYPT", "EGP", "EGP", "", ""),
            ("DE", "DEU", "GERMANY", "EUR", "EUR", "", ""),
            ("MX", "MEX", "MEXICO", "MXN", "MXN", "", ""),
            ("GB", "GBR", "UNITED KINGDOM", "GBP", "GBP", "", ""),
            ("BR", "BRA", "BRAZIL", "BRL", "BRL", "", "")
            ],
  StructType([
    StructField("CountryID", StringType()),
    StructField("CountryCode", StringType()),
    StructField("CountryDesc", StringType()),
    StructField("CtryLocalCurrencyCode", StringType()),
    StructField("CtryReportCurrencyCode", StringType()),
    StructField("MarketRegionCode", StringType()),
    StructField("MarketRegionDesc", StringType()),
])
    )
df_dim_country.write.mode('overwrite').format("delta").save(path + DIM_COUNTRY)

In [0]:
df_dim_distributors = spark.createDataFrame(
        [
          ("DHROP","BR","100","DISTRIBUICAO","280","","DISTRIBUICAO","ATIVO","HEADQUARTER","1"),
          ("MGROP","BR","101","DISTRIBUICAO","281","","DISTRIBUICAO","ATIVO","HEADQUARTER","1")
            ],
  StructType([
    StructField("DistributorID", StringType()),
    StructField("CountryID", StringType()),
    StructField("DistributorCode", StringType()),
    StructField("DistributorDesc", StringType()),
    StructField("DistributorAddress", StringType()),
    StructField("DistributorArea", StringType()),
    StructField("DistributorOwner", StringType()),
    StructField("Status", StringType()),
    StructField("HeadQuartersorBranch", StringType()),
    StructField("HeadQuarterDistributorID", StringType()),
])
    )
df_dim_distributors.write.mode('overwrite').format("delta").save(path + DIM_DISTRIBUTOR)

In [0]:
dim_pos = spark.createDataFrame(
        [
          ("111","BR","DHROP","PN",),
          ("222","BR","MGROP","PN",)
            ],
  StructType([
    StructField("POSCode", StringType()),
    StructField("CountryID", StringType()),
    StructField("DistributorID", StringType()),
    StructField("Segment", StringType())
])
    )
dim_pos.write.mode('overwrite').format("delta").partitionBy("CountryID","DistributorID","Segment").save(path + DIM_POS)

In [0]:
fact_sellout = spark.createDataFrame(
        [
          ("9999","111","20250101","1234","Yes","BRL","EA","10","Promo","150","20250102","BR","DHROP","PN",),
          ("8888","222","20250101","9876","Yes","BRL","EA","20","Promo","100","20250102","BR","MGROP","PN",)
            ],
  StructType([
    StructField("ProductCode", StringType()),
    StructField("POSCode", StringType()),
    StructField("InvoiceDate", StringType()),
    StructField("InvoiceCode", StringType()),
    StructField("InvoiceStatus", StringType()),
    StructField("CurrencyID", StringType()),
    StructField("UoMCode", StringType()),
    StructField("SoldProductQty", StringType()),
    StructField("Sales_Type", StringType()),
    StructField("SoldProductAmt", StringType()),
    StructField("LastModifiedDate", StringType()),
    StructField("CountryID", StringType()),
    StructField("DistributorID", StringType()),
    StructField("Segment", StringType())
])
    )
fact_sellout.write.mode('overwrite').format("delta").partitionBy("CountryID","DistributorID","Segment").save(path + FACT_SELLOUT)

In [0]:
df_country = spark.read.format("delta").load(path + DIM_COUNTRY)
df_distributor = spark.read.format("delta").load(path + DIM_DISTRIBUTOR)
df_pos = spark.read.format("delta").load(path + DIM_POS)
df_sellout = spark.read.format("delta").load(path + FACT_SELLOUT)

In [0]:
try:
    df_invoice = (
        df_sellout.join(df_country, ["CountryID"], "inner")
        .join(
            df_distributor,
            (df_sellout.CountryID == df_distributor.CountryID)
            & (df_sellout.DistributorID == df_distributor.DistributorID),
            "inner",
        )
        .select(
            df_sellout.ProductCode,
            df_distributor.DistributorID,
            df_country.CountryID,
            df_sellout.POSCode,
            df_sellout.InvoiceDate.alias("DateID"),
            df_sellout.InvoiceCode.alias("InvoiceNumber")
        )
        .distinct()
    )
except Exception as e:
    raise
    

In [0]:
df_invoice.display()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8169887414013658>, line 1
----> 1 df_invoice.display()

File /databricks/python_shell/lib/dbruntime/monkey_patches.py:52, in apply_dataframe_display_patch.<locals>.df_display(df, *args, **kwargs)
     48 def df_display(df, *args, **kwargs):
     49     """
     50     df.display() is an alias for display(df). Run help(display) for more information.
     51     """
---> 52     display(df, *args, **kwargs)

File /databricks/python_shell/lib/dbruntime/display.py:210, in Display.display(self, input, *args, **kwargs)
    208         self.display_connect_table(input)
    209 elif isinstance(input, ConnectDataFrame):
--> 210     if getattr(input, "isStreaming", False):
    211         handleStreamingConnectDataFrame(input, self.entry_point, kwargs)
    212     else:

File /usr/lib/python3.10/functools.py:981, in cached_property._

In [0]:
df_country = spark.read.format("delta").load(path + DIM_COUNTRY)
df_distributor = spark.read.format("delta").load(path + DIM_DISTRIBUTOR).filter((col("CountryID") == 'BR') & (col("DistributorID") == 'DHROP'))
df_pos = spark.read.format("delta").load(path + DIM_POS)
df_sellout = spark.read.format("delta").load(path + FACT_SELLOUT).filter(col("CountryID") == 'BR')

In [0]:
try:
    df_invoice = (
        df_sellout.join(df_country, ["CountryID"], "inner")
        .join(
            df_distributor,["CountryID"], "inner")
        .join(
            df_pos,["POSCode","CountryID","DistributorID"],
            "inner",
        )
        .select(
            df_sellout.ProductCode,
            df_sellout.POSCode,
            df_country.CountryID,
            df_distributor.DistributorID,
            df_sellout.POSCode,
            df_sellout.InvoiceDate.alias("DateID"),
            df_sellout.InvoiceCode.alias("InvoiceNumber")
        )
        .distinct()
    )
except Exception as e:
    raise

In [0]:
df_invoice.display()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3105691891063489>, line 1
----> 1 df_invoice.display()

File /databricks/python_shell/lib/dbruntime/monkey_patches.py:52, in apply_dataframe_display_patch.<locals>.df_display(df, *args, **kwargs)
     48 def df_display(df, *args, **kwargs):
     49     """
     50     df.display() is an alias for display(df). Run help(display) for more information.
     51     """
---> 52     display(df, *args, **kwargs)

File /databricks/python_shell/lib/dbruntime/display.py:215, in Display.display(self, input, *args, **kwargs)
    212     else:
    213         serialized_plan = base64.b64encode(
    214             input._plan.to_proto(input._session._client).SerializeToString()).decode()
--> 215         self.add_custom_display_data("table", serialized_plan)
    216 elif isinstance(input, list):
    217     self.display(self.sparkSession

### Luca's example end here

In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as pyf
from pyspark.sql.functions import udf, to_json, from_json, explode, explode_outer, col, array, array_contains, max as max_, min as min_, collect_list, collect_set, arrays_zip, lit, expr, sha2, concat_ws, size, struct, current_timestamp, current_date, sha2, concat_ws
from pyspark.sql.types import Row, ArrayType, StructType, MapType, StructField, StringType, TimestampType, DateType, IntegerType, LongType, DoubleType, BooleanType
import datetime as dt

In [0]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F

In [0]:
%sql
use catalog dkushari_uc;
use aaa_mars_schema;

In [0]:
%sql
select current_catalog();

current_catalog()
dkushari_uc


In [0]:
%sql
select current_schema();

current_schema()
aaa_mars_schema


In [0]:

df1 = spark.createDataFrame([
    [1, dt.date(2025,1,31), 'source1', 'cash_flow_id1', False],
    [2, dt.date(2025,1,30), 'source1', 'cash_flow_id1', False],
    [3, dt.date(2025,1,29), 'source1', 'cash_flow_id1', False],
    [4, dt.date(2025,1,28), 'source1', 'cash_flow_id1', False],
    [5, dt.date(2025,1,31), 'source1', 'cash_flow_id2', False],
    [6, dt.date(2025,1,30), 'source1', 'cash_flow_id2', False],
], StructType([
    StructField('scenario_id', IntegerType()), 
    StructField('business_date', DateType()),
    StructField('source', StringType()), 
    StructField('cash_flow_id', StringType()), 
    StructField('deleted', BooleanType())
]))

df1.write.mode('overwrite').option("path","abfss://dkushari@dkusharistorageaccount.dfs.core.windows.net/mars_2025/table1").saveAsTable('dkushari_uc.aaa_mars_schema.table1')

In [0]:
df2 = spark.createDataFrame([
    [10, dt.date(2025,10,31), 'source1', 'cash_flow_id1', False],
    [2, dt.date(2025,10,30), 'source1', 'cash_flow_id1', False],
    [30, dt.date(2025,10,29), 'source1', 'cash_flow_id1', False],
    [4, dt.date(2025,10,28), 'source1', 'cash_flow_id1', False],
    [50, dt.date(2025,10,31), 'source1', 'cash_flow_id2', False],
    [6, dt.date(2025,10,30), 'source1', 'cash_flow_id2', False],
], StructType([
    StructField('scenario_id', IntegerType()), 
    StructField('business_date', DateType()),
    StructField('source', StringType()), 
    StructField('cash_flow_id', StringType()), 
    StructField('deleted', BooleanType())
]))

df2.write.mode('overwrite').option("path","abfss://dkushari@dkusharistorageaccount.dfs.core.windows.net/mars_2025/table2").saveAsTable('dkushari_uc.aaa_mars_schema.table2')

In [0]:
df3 = spark.createDataFrame([
    [10, dt.date(2025,10,31), 'source1', 'cash_flow_id1', False],
    [20, dt.date(2025,10,30), 'source1', 'cash_flow_id1', False],
    [30, dt.date(2025,10,29), 'source1', 'cash_flow_id1', False],
    [40, dt.date(2025,10,28), 'source1', 'cash_flow_id1', False],
    [50, dt.date(2025,10,31), 'source1', 'cash_flow_id2', False],
    [6, dt.date(2025,10,30), 'source1', 'cash_flow_id2', False],
], StructType([
    StructField('scenario_id', IntegerType()), 
    StructField('business_date', DateType()),
    StructField('source', StringType()), 
    StructField('cash_flow_id', StringType()), 
    StructField('deleted', BooleanType())
]))

df3.write.mode('overwrite').option("path","abfss://dkushari@dkusharistorageaccount.dfs.core.windows.net/mars_2025/table3").saveAsTable('dkushari_uc.aaa_mars_schema.table3')

In [0]:
df_3=spark.read.load("abfss://dkushari@dkusharistorageaccount.dfs.core.windows.net/mars_2025/table3")
display(df_3)

In [0]:
df_1=spark.read.table("dkushari_uc.aaa_mars_schema.table1")
display(df_1)

scenario_id,business_date,source,cash_flow_id,deleted
1,2025-01-31,source1,cash_flow_id1,false
2,2025-01-30,source1,cash_flow_id1,false
3,2025-01-29,source1,cash_flow_id1,false
4,2025-01-28,source1,cash_flow_id1,false
5,2025-01-31,source1,cash_flow_id2,false
6,2025-01-30,source1,cash_flow_id2,false


In [0]:
df_2=spark.read.table("dkushari_uc.aaa_mars_schema.table2")
display(df_2)

scenario_id,business_date,source,cash_flow_id,deleted
10,2025-10-31,source1,cash_flow_id1,false
2,2025-10-30,source1,cash_flow_id1,false
30,2025-10-29,source1,cash_flow_id1,false
4,2025-10-28,source1,cash_flow_id1,false
50,2025-10-31,source1,cash_flow_id2,false
6,2025-10-30,source1,cash_flow_id2,false


In [0]:
df_3=spark.read.table("dkushari_uc.aaa_mars_schema.table2")
display(df_3)

scenario_id,business_date,source,cash_flow_id,deleted
10,2025-10-31,source1,cash_flow_id1,false
2,2025-10-30,source1,cash_flow_id1,false
30,2025-10-29,source1,cash_flow_id1,false
4,2025-10-28,source1,cash_flow_id1,false
50,2025-10-31,source1,cash_flow_id2,false
6,2025-10-30,source1,cash_flow_id2,false


In [0]:
df_test1 = df_1.join(df_2, ["scenario_id"], "inner")
display(df_test1)

scenario_id,business_date,source,cash_flow_id,deleted,business_date,source,cash_flow_id,deleted
2,2025-01-30,source1,cash_flow_id1,false,2025-10-30,source1,cash_flow_id1,false
4,2025-01-28,source1,cash_flow_id1,false,2025-10-28,source1,cash_flow_id1,false
6,2025-01-30,source1,cash_flow_id2,false,2025-10-30,source1,cash_flow_id2,false


In [0]:
df_test2 = df_1.join(df_2, ["scenario_id","cash_flow_id"], "inner").join(df_3, ["scenario_id"], "inner").select("scenario_id","cash_flow_id")
display(df_test2)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1332008936126529>, line 1
----> 1 df_test2 = df_1.join(df_2, ["scenario_id","cash_flow_id"], "inner").join(df_3, ["scenario_id"], "inner").select("scenario_id","cash_flow_id")
      2 display(df_test2)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3832, in DataFrame.select(self, *cols)
   3787 def select(self, *cols: "ColumnOrName") -> "DataFrame":  # type: ignore[misc]
   3788     """Projects a set of expressions and returns a new :class:`DataFra

In [0]:
df_test3 = df_1.join((df_1.scenario_id == df_2.scenario_id) & (df_1.cash_flow_id == df_2.cash_flow_id), "inner").join((df_2.scenario_id == df_3.scenario_id),"inner").select(df_1.scenario_id,df_1.cash_flow_id)

In [0]:
df_1=spark.read.load("abfss://dkushari@dkusharistorageaccount.dfs.core.windows.net/mars_2025/table1")
display(df_1)

scenario_id,business_date,source,cash_flow_id,deleted
1,2025-01-31,source1,cash_flow_id1,false
2,2025-01-30,source1,cash_flow_id1,false
3,2025-01-29,source1,cash_flow_id1,false
4,2025-01-28,source1,cash_flow_id1,false
5,2025-01-31,source1,cash_flow_id2,false
6,2025-01-30,source1,cash_flow_id2,false


In [0]:
df_2=spark.read.load("abfss://dkushari@dkusharistorageaccount.dfs.core.windows.net/mars_2025/table2")
display(df_2)

scenario_id,business_date,source,cash_flow_id,deleted
10,2025-10-31,source1,cash_flow_id1,false
2,2025-10-30,source1,cash_flow_id1,false
30,2025-10-29,source1,cash_flow_id1,false
4,2025-10-28,source1,cash_flow_id1,false
50,2025-10-31,source1,cash_flow_id2,false
6,2025-10-30,source1,cash_flow_id2,false


In [0]:
df_3=spark.read.load("abfss://dkushari@dkusharistorageaccount.dfs.core.windows.net/mars_2025/table3")
display(df_3)

scenario_id,business_date,source,cash_flow_id,deleted
10,2025-10-31,source1,cash_flow_id1,false
20,2025-10-30,source1,cash_flow_id1,false
30,2025-10-29,source1,cash_flow_id1,false
40,2025-10-28,source1,cash_flow_id1,false
50,2025-10-31,source1,cash_flow_id2,false
6,2025-10-30,source1,cash_flow_id2,false


In [0]:
df_test2 = df_1.join(df_2, ["scenario_id"], "inner").join(df_3, ["scenario_id"], "inner").select("scenario_id","cash_flow_id")
display(df_test2)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1332008936126752>, line 1
----> 1 df_test2 = df_1.join(df_2, ["scenario_id"], "inner").join(df_3, ["scenario_id"], "inner").select("scenario_id","cash_flow_id")
      2 display(df_test2)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3832, in DataFrame.select(self, *cols)
   3787 def select(self, *cols: "ColumnOrName") -> "DataFrame":  # type: ignore[misc]
   3788     """Projects a set of expressions and returns a new :class:`DataFrame`.
   3789 
 

In [0]:
df_2.groupBy("cash_flow_id").agg(F.max("business_date").alias("max")).display()

cash_flow_id,max
cash_flow_id1,2025-10-31
cash_flow_id2,2025-10-31


In [0]:

tbl1 = DeltaTable.forName(spark, 'table1')
df1=tbl1.toDF()
display(df1)

scenario_id,business_date,source,cash_flow_id,deleted
1,2025-01-31,source1,cash_flow_id1,false
2,2025-01-30,source1,cash_flow_id1,false
3,2025-01-29,source1,cash_flow_id1,false
4,2025-01-28,source1,cash_flow_id1,false
5,2025-01-31,source1,cash_flow_id2,false
6,2025-01-30,source1,cash_flow_id2,false


In [0]:
tbl2 = DeltaTable.forName(spark, 'table2')
df2=tbl2.toDF()
display(df2)

scenario_id,business_date,source,cash_flow_id,deleted
10,2025-10-31,source1,cash_flow_id1,false
2,2025-10-30,source1,cash_flow_id1,false
30,2025-10-29,source1,cash_flow_id1,false
4,2025-10-28,source1,cash_flow_id1,false
50,2025-10-31,source1,cash_flow_id2,false
6,2025-10-30,source1,cash_flow_id2,false


In [0]:
df_test1 = df1.join(df2, ["scenario_id"], "inner") # IT FAILS ON UC CLUSTER
#df_test2 = df1.alias('a').join(df2.alias('b'), cond2, 'left') 

In [0]:
display(df_test1)

scenario_id,business_date,source,cash_flow_id,deleted,source,cash_flow_id,deleted
1,2025-01-31,source1,cash_flow_id1,false,null,null,null
2,2025-01-30,source1,cash_flow_id1,false,null,null,null
3,2025-01-29,source1,cash_flow_id1,false,null,null,null
4,2025-01-28,source1,cash_flow_id1,false,null,null,null
5,2025-01-31,source1,cash_flow_id2,false,null,null,null
6,2025-01-30,source1,cash_flow_id2,false,null,null,null


In [0]:
df1 = spark.read.table("table1")
df2 = spark.read.table("table2")

In [0]:
df_test1 = df1.join(df2, ["scenario_id"], "inner") 
display(df_test1)

scenario_id,business_date,source,cash_flow_id,deleted,business_date,source,cash_flow_id,deleted
2,2025-01-30,source1,cash_flow_id1,false,2025-10-30,source1,cash_flow_id1,false
4,2025-01-28,source1,cash_flow_id1,false,2025-10-28,source1,cash_flow_id1,false
6,2025-01-30,source1,cash_flow_id2,false,2025-10-30,source1,cash_flow_id2,false


In [0]:
path="abfss://dkushari@dkusharistorageaccount.dfs.core.windows.net/mars_2025/table1"

In [0]:
tbl = DeltaTable.forPath(spark, path)
tbl.toDF().groupBy("cash_flow_id").agg(F.max("business_date").alias("max")).display()

cash_flow_id,max
cash_flow_id1,2025-01-31
cash_flow_id2,2025-01-31


In [0]:

def filter_valuations_by_scenarios_and_date(scenario_ids, business_date, val):
    df = val
    df = df.filter(df.scenario_id.isin(scenario_ids) & (df.business_date <= business_date))
    window = Window.partitionBy('source', 'cash_flow_id').orderBy(col('business_date').desc())
    df = df.withColumn('row', pyf.row_number().over(window))
    df = df.filter(~df.deleted & (df.row == 1)).drop('row')
    df = df.withColumn('business_date', lit(business_date))
    return df

In [0]:
df = filter_valuations_by_scenarios_and_date([1,2,3], dt.date(2025,1,29), val)
display(df)

scenario_id,business_date,source,cash_flow_id,deleted
1,2025-01-29,source1,cash_flow_id1,false


In [0]:
val_df = spark.read.table("00535837_test1_00600401_table")
df = filter_valuations_by_scenarios_and_date([1,2,3], dt.date(2025,1,29), val_df)
display(df)

scenario_id,business_date,source,cash_flow_id,deleted
1,2025-01-29,source1,cash_flow_id1,false


In [0]:
#################
# This will fail on a mult-node shared cluster but will not fail on a single-node single user cluster
####################
from delta.tables import DeltaTable
val_table = DeltaTable.forName(spark, "00535837_test1_00600401_table")
df = filter_valuations_by_scenarios_and_date([1,2,3], dt.date(2025,1,29), val_table.toDF())
display(df)

scenario_id,business_date,source,cash_flow_id,deleted
1,2025-01-29,source1,cash_flow_id1,false
